# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [6]:
# #langchain library
!pip install langchain==0.1.6
!pip install langchain-community==0.0.19
# #LLM
!pip install accelerate==0.25.0
!pip install transformers==4.36.2
!pip install bitsandbytes==0.43
# #Text Embedding
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding==1.0.1
# #vectorstore
!pip install pymupdf==1.23.8
!pip install faiss-gpu==1.7.2
!pip install faiss-cpu==1.7.4

In [7]:
import os
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [8]:
from langchain.prompts import PromptTemplate

prompt_template = """
    Hello there! I'm AITBot, your helpful virtual assistant ready to answer any queries
    you might have about the Asian Institute of Technology (AIT).
    Whether you're seeking information about AIT, its array of programs and courses,
    or any related topic, feel free to ask away!
    Simply share your questions with me,
    and I'll provide you with the guidance you need.
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="Hello there! I'm AITBot, your helpful virtual assistant ready to answer any queries\n    you might have about the Asian Institute of Technology (AIT).\n    Whether you're seeking information about AIT, its array of programs and courses,\n    or any related topic, feel free to ask away!\n    Simply share your questions with me,\n    and I'll provide you with the guidance you need.\n    {context}\n    Question: {question}\n    Answer:")

In [9]:
PROMPT.format(
    context = "The Asian Institute of Technology (AIT) is an international English-speaking postgraduate institution",
    question = "What is ait?"
)

"Hello there! I'm AITBot, your helpful virtual assistant ready to answer any queries\n    you might have about the Asian Institute of Technology (AIT).\n    Whether you're seeking information about AIT, its array of programs and courses,\n    or any related topic, feel free to ask away!\n    Simply share your questions with me,\n    and I'll provide you with the guidance you need.\n    The Asian Institute of Technology (AIT) is an international English-speaking postgraduate institution\n    Question: What is ait?\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code).
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [10]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = 'AIT-BROCHURE.pdf'
loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [11]:
# documents

In [12]:
len(documents)

2

In [13]:
documents[1]

Document(page_content='Department of Food, \nAgriculture and Bioresources\nAgribusiness Management \nAgricultural Systems and \nEngineering \nAquaculture and Aquatic \nResources Management \nFood Engineering and \nBioprocess Technology \nFood Innovation, Nutrition and \nHealth\nS T U DY  O P T I O N S  A T  A I T\nMaster’s and PhD Degree Programs\nAvailable in many fields of specializations across its \nthree Schools, covering various aspects of techno-\nlogy, social developments, and management. Sev-\neral new degree programs are designed to address \nemerging global challenges and to respond to cur-\nrent demands of the workforce. \nFlexible Degree Options\nNew alternative to traditional “On-campus Degree \nOptions”. Students can do many courses online via \nAIT’s Hybrid Modes of Instruction, with an option of \nflexible payment: course-by-course. The minimum \nenrollment is two courses per semester – ideal for those \nnot wanting to take too much time away from career, \nor those wa

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [15]:
doc[2]

Document(page_content='Yunus Center at AIT\n- - E nte r p r i s e  s o l u t i o n s fo r  S D G s & \ntechnologies to alleviate poverty; member \nof global Yunus network\nAIT Center Vietnam\n- -Provisions of academic programs, \ntraining, and consulting in Vietnam\nO U T R E A C H\nC E N T E R S\nMr. Chaiwat Kovavisarash \n(Class of 1991)\nCurrent Position \nCEO, President & Director, Bangchak Corporation \nPublic Company Limited, Thailand\nAIT opened up my mind exposing me to in-\nternational surroundings and transforming \nme to be a mature and lifelong learning \nperson. It was this memorable 20-month \nprocess that has taken me thus far.\nD r.  B i n d a  P a n d e y \n(Class of 2002)\nCurrent Position', metadata={'source': 'AIT-BROCHURE.pdf', 'file_path': 'AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Build 20G165) Quartz PDFCo

In [16]:
len(doc)

25

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [17]:
import torch
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [18]:
#locate vectorstore
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

create path done


In [19]:
#save vector locally
from langchain_community.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_ait'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [20]:
#calling vector from local
vector_path = './vector-store'
db_file_name = 'nlp_ait'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [21]:
#ready to use
retriever = vectordb.as_retriever()

In [22]:
retriever.get_relevant_documents("What are the courses at AIT?")

[Document(page_content='AIT offers an opportunity to all students \nenrolling in Master’s degree programs \nto take internships in national and \nmultinational enterprises as a part of \nthe academic curriculum. The internship \ncarries 12 credits and duration can \nrange from 4 to 6 months. The internship \nwork generally relates to the academic \nprogram and the research area of the \nstudents, and includes student-reports \non the work done during the internship. \nThis provides the students practical \nexperiences, exposure, networking and \njob placement opportunities in future.\nMinor \nSeveral programs at AIT also offer minors. \nA minor will complement the knowledge \nacquired in the major program and can', metadata={'source': 'AIT-BROCHURE.pdf', 'file_path': 'AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Build 20G165) Quartz

In [23]:
retriever.get_relevant_documents("Where is AIT located?")

[Document(page_content='Apart from academic programs in three \nAIT Schools, these Outreach Centers are \nthe connectors for AIT’s education and re-\nsearch outreach with industry and society.\nAIT Extension\n--Executive education, lifelong learning, skill \ndevelopment through short-courses, trainings, \nworkshops, and seminars\nAIT Solutions\n- - Innovative consulting/solutions in \nengineering, technology, environment, \ninfrastructure, education and management\nGeoinformatics Center \n--Consulting and capacity building in \napplication of Remote Sensing and \nGeoinformatics in agriculture, disaster risk \nmanagement, and environment\nRegional Resource Center for Asia \nand Pacific \n--Advancement of Region’s environment', metadata={'source': 'AIT-BROCHURE.pdf', 'file_path': 'AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Build 20G

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [24]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [25]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [26]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [27]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [28]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [29]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [30]:
%cd ./models
!git clone https://huggingface.co/gavin124/gpt2-finetuned-cnn-summarization-v1

[Errno 2] No such file or directory: './models'
/content
Cloning into 'gpt2-finetuned-cnn-summarization-v1'...
remote: Enumerating objects: 20, done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (20/20), 523.58 KiB | 6.38 MiB/s, done.


In [34]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = 'gpt2-finetuned-cnn-summarization-v1'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'cuda:0',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'S

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [35]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [36]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [37]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [38]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': 'Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\nHuman:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:\nFollow Up Input: Comparing both of them\nStandalone question:\nWhat is Machine Learning?\nAI:\nFollow Up:\nWhat is Deep Learning?\nFollow Up:'}

`combine_docs_chain`

In [39]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="Hello there! I'm AITBot, your helpful virtual assistant ready to answer any queries\n    you might have about the Asian Institute of Technology (AIT).\n    Whether you're seeking information about AIT, its array of programs and courses,\n    or any related topic, feel free to ask away!\n    Simply share your questions with me,\n    and I'll provide you with the guidance you need.\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7f935c5e3910>)), document_variable_name='context')

In [40]:
query = "What is Transformers?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello there! I'm AITBot, your helpful virtual assistant ready to answer any queries
    you might have about the Asian Institute of Technology (AIT).
    Whether you're seeking information about AIT, its array of programs and courses,
    or any related topic, feel free to ask away!
    Simply share your questions with me,
    and I'll provide you with the guidance you need.
    Regenerative Sanitation 
Sustainable Energy Transition
Department of Development  
and Sustainability
Development and Sustainability 
Gender and Development 
Studies 
Natural Resources Management 
Development Planning 
Management and Innovation 
Urban Innovation and 
Sustainability 
Urban Sustainability Planning 
and Design 
Society and Environmental 
Governance 
Department of Civil and 
Infrastructure Engineering
Construction Engineering and 
Infrastructure Management 
Geotechnical and Earth 
Resources Eng

{'input_documents': [Document(page_content='Regenerative Sanitation \nSustainable Energy Transition\nDepartment of Development  \nand Sustainability\nDevelopment and Sustainability \nGender and Development \nStudies \nNatural Resources Management \nDevelopment Planning \nManagement and Innovation \nUrban Innovation and \nSustainability \nUrban Sustainability Planning \nand Design \nSociety and Environmental \nGovernance \nDepartment of Civil and \nInfrastructure Engineering\nConstruction Engineering and \nInfrastructure Management \nGeotechnical and Earth \nResources Engineering \nStructural Engineering \nTransportation Engineering \nWater Engineering and \nManagement \nDepartment of Information \nand Communication \nTechnologies \nComputer Science', metadata={'source': 'AIT-BROCHURE.pdf', 'file_path': 'AIT-BROCHURE.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macO

In [41]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="Hello there! I'm AITBot, your helpful virtual assistant ready to answer any queries\n    you might have about the Asian Institute of Technology (AIT).\n    Whether you're seeking information about AIT, its array of programs and courses,\n    or any related topic, feel free to ask away!\n    Simply share your questions with me,\n    and I'll provide you with the guidance you need.\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7f935c5e3910>)), document_variable_name='context'), question_generator=LLMChain(verbose=True, prompt=PromptTemplate(input_vari

## 5. Chatbot

In [42]:
prompt_question = "Can you tell me about AIT?"
answer = chain({"question":prompt_question})
answer


> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'Can you tell me about AIT?',
 'chat_history': [],
 'answer': "Hello there! I'm AITBot, your helpful virtual assistant ready to answer any queries\n    you might have about the Asian Institute of Technology (AIT).\n    Whether you're seeking information about AIT, its array of programs and courses,\n    or any related topic, feel free to ask away!\n    Simply share your questions with me,\n    and I'll provide you with the guidance you need.\n    academic, research, and outreach programs \nprepare graduates for professional success and \nleadership roles in Asia and beyond. \nFounded in 1959, AIT offers students the \nopportunity to study at an institution in Asia \nwhich possesses a global reputation. AIT stresses \nits international connections, its injection of \ninnovation into research and teaching, its \nengagement with public and private sectors, \nand its nurturing of entrepreneurship. AIT \ncontinues to fulfill its capacity-building role \nand has oriented its edu

In [44]:
prompt_question = "What are the scholarships that AIT is offering?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Can you tell me about AIT?'), AIMessage(content="Hello there! I'm AITBot, your helpful virtual assistant ready to answer any queries\n    you might have about the Asian Institute of Technology (AIT).\n    Whether you're seeking information about AIT, its array of programs and courses,\n    or any related topic, feel free to ask away!\n    Simply share your questions with me,\n    and I'll provide you with the guidance you need.\n    academic, research, and outreach programs \nprepare graduates for professional success and \nleadership roles in Asia and beyond. \nFounded in 1959, AIT offers students the \nopportunity to study at an institution in Asia \nwhich possesses a global reputation. AIT

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
